In [6]:
import mido as md
import numpy as np

In [10]:
# Outputs a list of transition matrices (128x128), one for each track in the MIDI whose filepath is input
def generate_transition_probs(filepath):
    poops = []
    song = md.MidiFile(filepath)
    
    for track in song.tracks:
        poop = np.zeros((128, 128))
        prevnote = None
        n = np.zeros(128)
        for msg in track:
            x = vars(msg)
            if x['type'] == 'note_on' and x['velocity'] != 0:
                note = x['note']
                if prevnote is not None:
                    poop[prevnote][note] += 1
                    n[prevnote] += 1
                prevnote = note

        for i in range(128):
            if (n[i] != 0):
                poop[i] /= n[i]
        poops.append(poop)
    return poops

In [11]:
# Adds an integer offset to all the notes of the MIDI whose filepath is input
def transpose(filepath, offset):
    result = md.MidiFile()
    
    song = md.MidiFile(filepath)
    for track in song.tracks:
        newtrack = md.MidiTrack()
        result.tracks.append(newtrack)
        for msg in track:
            x = vars(msg)
            if x['type'] == 'note_on':
                note = x['note']
                note += offset
                newmessage = md.Message('note_on', note=note, velocity=x['velocity'], time=x['time'])
                newtrack.append(newmessage)
            else:
                newtrack.append(msg)
    result.save("midis/1.mid")

In [12]:
def findKeySignature(song):
    for i, track in enumerate(song.tracks):
        for msg in track:
            if msg.type == 'key_signature':
                return msg.key
            
# Finds the offset needed to transpose a MIDI to C major            
def findTransposeOffset(filepath):
    song = md.MidiFile(filepath)
    keys = ['C', 'C#/Db', 'D', 'D#/Eb', 'E', 'F', 'F#/Gb', 'G', 'G#/Ab', 'A', 'A#/Bb', 'B']
    keySig = findKeySignature(song)
    print(keySig)
    for i in range(len(keys)):
        if keySig in keys[i]:
            offset = i
            break
    return offset

In [206]:
# Returns the note-to-note transitions and
# the notelength-to-notelength transitions.

# Uncomment the line which sets max_len to 2000 if needed, and increase 2000 if needed
def generate_transition_probs_and_times(filepath):
    poops = []
    times = []
    max_len = 0
    song = md.MidiFile(filepath)
    for track in song.tracks:
        for msg in track:
            if msg.type == 'note_on':
                max_len = max(max_len, msg.time)
    #max_len=2000
    for track in song.tracks:
        time_matrix = np.zeros((max_len+1, max_len+1))
        poop = np.zeros((128, 128))
        prevnote = None
        prevlength = None
        curr_length = 0
        n = np.zeros(128)
        n_times = np.zeros(max_len+1)
        for msg in track:
            x = vars(msg)
            if x['type'] == 'note_on' and x['velocity'] != 0:
                note = x['note']
                curr_length += x['time']
                if prevnote is not None:
                    poop[prevnote][note] += 1
                    n[prevnote] += 1
                if prevlength is not None:
                    time_matrix[prevlength][curr_length] +=1
                    n_times[prevlength] += 1
                prevnote = note
                prevlength = curr_length
                curr_length = 0
            elif x['type'] == 'note_on':
                curr_length += x['time']
        for i in range(128):
            if n[i] != 0:
                poop[i] /= n[i]
        for j in range(max_len+1):
            if n_times[j] != 0:
                time_matrix[j] /= n_times[j]
        poops.append(poop)
        times.append(time_matrix)
    return poops, times

In [207]:
songLength = 500
starting_note = 66
pMatrices, times = generate_transition_probs_and_times("midis/chpn_op25_e11.mid")

In [209]:
# Generate a random track based on the 2th track (left hand)
# of Chopin's Winter Wind with note-transition probs
# AND note-length transition probs
# (EXPERIMENT 3)

from mido import Message, MidiFile, MidiTrack

delta = 80
current_note = starting_note
previous_note = starting_note
current_length = 120
previous_length = current_length
current_time = 0

rh_trans = pMatrices[2]
rh_times = times[2]
print(rh_times.shape)
# print(rh_trans[current_note])

outfile = MidiFile()

track = MidiTrack()
outfile.tracks.append(track)

# track.append(Message('instrument_name', 'piano'))
track.append(Message('program_change', program=12))

hardcap = 480
for i in range(songLength):
    track.append(Message('note_on', note = current_note, velocity = 127, time = current_length))
    
    possibleNexts = {i:rh_trans[current_note][i] for i in range(128) if rh_trans[current_note][i] > 0}
    notes = list(possibleNexts.keys())
    probabilities = [possibleNexts[n] for n in notes]
    
    possibleNextLengths = {i:rh_times[current_length][i] for i in range(len(rh_times)) if rh_times[current_length][i] > 0}
    lengths = list(possibleNextLengths.keys())
    prob_lengths = [possibleNextLengths[n] for n in lengths]
    
    previous_note = current_note
    previous_length = current_length
    
    current_note = np.random.choice(notes, 1, p=probabilities)[0]
    if len(lengths) < 1: 
        x=1/0
        print(current_length)
    current_length = np.random.choice(lengths, size=1, p=prob_lengths)[0]

outfile.save('winterwindLH.mid')

(3841, 3841)


In [3]:
# Simple single-track-based generation. (EXPERIMENT 1)
# Output a single track whose note-to-note transition probabilities 
# are based off of Chopin's Black Keys

import sys
import mido as md 
from mido import MidiFile
from collections import defaultdict
import numpy as np
from sklearn.preprocessing import normalize

song = md.MidiFile('midis/chopinblack.mid')
trackNotes = defaultdict(list)
pMatrices = defaultdict(list)
for i, track in enumerate(song.tracks):
    pMatrix = np.zeros((128, 128))
    prevNote = None
    for msg in track:
        if msg.type == 'note_on' and msg.velocity:
            if prevNote:
                pMatrix[prevNote][msg.note] += 1
            prevNote = msg.note
    if pMatrix.any():
        pMatrices[i] = normalize(pMatrix, axis =1, norm = 'l1')
songLength = 500
starting_note = 66

from mido import Message, MidiFile, MidiTrack, MetaMessage

delta = 600
current_note = starting_note
current_note2 = starting_note + 5
current_time = 0

rh_trans = pMatrices[1]
lh_trans = pMatrices[2]

# print(rh_trans[current_note])

outfile = MidiFile()

track = MidiTrack()
track2 = MidiTrack()
outfile.tracks.append(track)
outfile.tracks.append(track2)

# track.append(Message('instrument_name', 'piano'))
track.append(Message('program_change', program=12))

track.append(MetaMessage(type = "track_name", name = "Piano right", time = 0))
for i in range(songLength):
    track.append(Message('note_on', note = current_note, velocity = 127, time = 75))
#     track.append(Message('note_on', note = current_note, velocity = 0, time = 40))
    possibleNexts = {i:rh_trans[current_note][i] for i in range(128) if rh_trans[current_note][i] > 0}
    notes = list(possibleNexts.keys())
    probabilities = [possibleNexts[n] for n in notes]
    current_note = np.random.choice(notes, 1, p=probabilities)[0]
track.append(MetaMessage(type = 'end_of_track', time = 0))

track2.append(MetaMessage(type = 'track_name', name = "Piano left", time = 60))
for i in range(songLength):
    track2.append(Message('note_on', note = current_note2, velocity = 60, time = 75))
    possibleNexts = {i:lh_trans[current_note2][i] for i in range(128) if lh_trans[current_note2][i] > 0}
    notes = list(possibleNexts.keys())
    probabilities = [possibleNexts[n] for n in notes]
    current_note2 = np.random.choice(notes, 1, p=probabilities)[0]
track2.append(MetaMessage(type = 'end_of_track', time = 0))


outfile.save('testChopin.mid')
